In [1]:
import os
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

import mne.time_frequency
from ieeg.viz.ensemble import chan_grid
from ieeg.viz.parula import parula_map
from ieeg.io import get_data, update, get_bad_chans
import os
import matplotlib.pyplot as plt
from src.analysis.spec.wavelet_functions import load_wavelets
from src.analysis.utils.general_utils import make_or_load_subjects_electrodes_to_ROIs_dict, get_sig_chans_per_subject, make_sig_electrodes_per_subject_and_roi_dict

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


plots just the rescaled wavelets 2/28/25

In [ ]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

subjects = ['D0057']

# define a list of epochs root files that you want to plot wavelets for
epochs_root_files_for_wavelets = ['Stimulus_fixationCrossBase_0.2sec']

rescaled=True

for sub in subjects:
    layout = get_data("GlobalLocal", root=LAB_root)

    fig_path = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', 'figs')

    for epochs_root_file in epochs_root_files_for_wavelets:
        spec = load_wavelets(sub, layout, epochs_root_file, rescaled)
            
        info_file = os.path.join(layout.root, spec.info['subject_info']['files'][0])

        # Check channels for outliers and remove them
        all_bad = get_bad_chans(info_file)
        spec.info.update(bads=[b for b in all_bad if b in spec.ch_names])

        # Plotting
        figs = chan_grid(spec, size=(20, 10), vmin=-2, vmax=2, cmap=parula_map, show=False)

        for i, f in enumerate(figs):
            if rescaled:
                fig_name = f'{sub}_{output_name}_rescaled_{i+1}.jpg'
            else:
                fig_name = f'{sub}_{output_name}_uncorrected_{i+1}.jpg'

            fig_pathname = os.path.join(fig_path, fig_name)
            f.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\wavelet\D0057\Stimulus_s25and75_fixationCrossBase_0.5sec_rescaled-tfr.h5 ...
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No baseline correction applied
No base

let's plot wavelets for a specific roi, either sig or all

In [5]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

subjects = ['D0059', 'D0063']

# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

epochs_root_files_for_sig_elecs = ["Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10", 
"Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False", 
"Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False", 
"Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"]

epochs_root_files_for_wavelets = ['Stimulus_fixationCrossBase_0.2sec']

rescaled=True

layout = get_data("GlobalLocal", root=LAB_root)
fig_path = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', 'figs')

for epochs_root_file_for_sig_elecs in epochs_root_files_for_sig_elecs:
    # Load the significant channels for each subject based on the above condition
    sig_chans_per_subject = get_sig_chans_per_subject(
        subjects, epochs_root_file=epochs_root_file_for_sig_elecs, LAB_root=LAB_root
    )

    electrodes_per_subject_roi, sig_electrodes_per_subject_roi = make_sig_electrodes_per_subject_and_roi_dict(
        rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
    )

    for sub in subjects:
        for epochs_root_file_for_wavelets in epochs_root_files_for_wavelets:
            spec = load_wavelets(sub, layout, epochs_root_file_for_wavelets, rescaled)
            info_file = os.path.join(layout.root, spec.info['subject_info']['files'][0])
            # Check channels for outliers and remove them
            all_bad = get_bad_chans(info_file)

            for roi in rois_dict.keys():
                # Get the significant channels and all channels for this subject and ROI
                sig_chans_in_roi = sig_electrodes_per_subject_roi[roi].get(sub, [])
                all_chans_in_roi = electrodes_per_subject_roi[roi].get(sub, [])
                
                # define the two plotting jobs we need to run for this roi
                plot_jobs = {
                    'all_elecs': all_chans_in_roi,
                    'sig_elecs': sig_chans_in_roi
                }
                
                # loop through both 'all_elecs' and 'sig_elecs' jobs
                for plot_type, channel_list in plot_jobs.items():
                    if not channel_list:
                        print(f"No channels found for {plot_type} in {roi} for {sub}")
                        continue

                    # Filter list to channels that exist in the wavelet data
                    chans_to_plot = [ch for ch in channel_list if ch in spec.ch_names]

                    if not chans_to_plot:
                        print(f"Info: No channels for {plot_type} in {roi} found in the data. Skipping.")
                        continue

                    spec_roi = spec.copy().pick(chans_to_plot)
                    spec_roi.info.update(bads=[b for b in all_bad if b in spec_roi.ch_names])

                    # Plotting
                    print(f"Plotting {sub}, ROI: {roi}, Type: {plot_type} ({len(spec_roi.ch_names)} channels)")

                    # Plotting
                    figs = chan_grid(spec_roi, size=(20, 10), vmin=-2, vmax=2, cmap=parula_map, show=False)

                    for i, f in enumerate(figs):
                        
                        rescaled_str = 'rescaled' if rescaled else 'uncorrected'

                        # Create a descriptive filename that includes the plot type
                        fig_name = (
                            f'{sub}_{epochs_root_file_for_sig_elecs}_{roi}_{plot_type}_{epochs_root_file_for_wavelets}_wavelets'
                            f'_{rescaled_str}_{i+1}.jpg'
                        )

                        fig_pathname = os.path.join(fig_path, fig_name)
                        f.savefig(fig_pathname, bbox_inches='tight')
                        plt.close(f)  # Close figure to free memory
                        print(f"Saved figure: {fig_name}")        


Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: 